In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

os.listdir('/kaggle/input')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()
import re

In [ ]:
def split_form(form):
    string = ''
    for i in re.findall(r"[A-Z][^A-Z]*", form):
        elem = re.match(r"\D+", i).group()
        num = i.replace(elem, "")
        if num == "":
            string += f"{elem} "
        else:
            string += f"{elem} {str(num)} "
    return string.rstrip(' ')
def split_form2(form):
    string = ''
    for i in re.findall(r"[a-z][^a-z]*", form):
        elem = i[0]
        num = i.replace(elem, "").replace('/', "")
        num_string = ''
        for j in re.findall(r"[0-9]+[^0-9]*", num):
            num_list = list(re.findall(r'\d+', j))
            assert len(num_list) == 1, f"len(num_list) != 1"
            _num = num_list[0]
            if j == _num:
                num_string += f"{_num} "
            else:
                extra = j.replace(_num, "")
                num_string += f"{_num} {' '.join(list(extra))} "
        string += f"/{elem} {num_string}"
    return string.rstrip(' ')

In [ ]:
import Levenshtein
from time import time
import pickle
# range more beautiful
from PIL import Image
import math
# nn module
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
import torch.nn.functional as F
import cv2

os.chdir('/kaggle/input/bms-molecular-translation')
os.listdir()

In [ ]:
def string_to_ints(string,max_len=100):
    l=[word2num['<BOS>']]
    if len(string) > max_len:
        string = string[:max_len]
    for s in string:
        l.append(word2num[s])
    l.append(word2num['<EOS>'])
    for miss_num in range(max_len - len(string)):
        l.append(word2num['<PAD>'])
    return l
def string_to_ints_comp(string,max_len=100):
    l = [word2num['<BOS>']]
    #string = list(string)
    if len(string) > max_len:
        string = string[:max_len]
    for s in string:
        l.append(word2num[s])
    l.append(word2num['<EOS>'])
    for miss_num in range(max_len - len(string)):
        l.append(word2num['<PAD>'])
    return l
def ints_to_string(l):
    l = np.array(l)
    l = l[l<36]
    return "InChI=1S/"+''.join(list(map(lambda i:num2word[i] ,l)))
def ints_to_string_comp(l):
    l = np.array(l)
    l = l[l<190]
    return "InChI=1S/"+''.join(list(map(lambda i:num2word[i] ,l)))
def idgetpath(idv,type1='train'):
    #type1 = 'train' or 'test'
    #path_att = "/kaggle/input/bms-molecular-translation/"
    path = type1+'/{}/{}/{}/{}.png'.format(idv[0],idv[1],idv[2],idv)
    #print(path)
    return path
def read_pic2(path):
    f = cv2.imread(path)
    return inference_transform(f)

In [ ]:
CFG_train = True
if CFG_train:
    train = pd.read_csv('/kaggle/input/bms-molecular-translation/train_labels.csv')
    train['InChI_1'] = train['InChI'].progress_apply(lambda x: x.split('/')[1])
    train['InChI_text'] = train['InChI_1'].progress_apply(split_form) + ' ' + \
                                train['InChI'].apply(lambda x: '/'.join(x.split('/')[2:])).progress_apply(split_form2).values
    tr_label = train
    tr_label['filepath'] = tr_label['image_id'].progress_apply(idgetpath)

In [ ]:
datadir = '/kaggle/input/model-data/'
word2num = pickle.load(open(datadir+'word2num_comp.pkl','rb'))
num2word = pickle.load(open(datadir+'num2word_comp.pkl','rb'))

In [ ]:
import torchvision.transforms as transforms
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]
inference_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

In [ ]:
import math

class input_pic(Dataset):
    def __init__(self,path,label,maxlen=160):
        self.path = path
        self.loader = read_pic2
        self.label = label
        self.maxlen = maxlen
    def __len__(self):
        return len(self.path)
    def __getitem__(self,index):
        sample = self.loader(self.path[index])
        label = string_to_ints_comp(self.label[index].split(),self.maxlen)
        return sample,torch.LongTensor(label),index
class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        resnet = models.resnet34(pretrained=True)
        modules = list(resnet.children())[:-2]
        self.resnet = nn.Sequential(*modules)
        for param in self.resnet.parameters():
            param.requires_grad = True
            
    def forward(self,x):
        x = self.resnet(x)
        x = x.permute(0,2,3,1)#chan is the last it is 512
        x = x.view(x.size(0),-1,x.size(-1))
        return x.permute(1,0,2)
"""   
class EncoderCNN(nn.Module):
    def __init__(self, embed_size, batch_size=64,res_grad=False,prtrain=True):
        super(EncoderCNN, self).__init__()
        resnet = models.resnet18(pretrained=prtrain)
        for param in resnet.parameters():
            param.requires_grad_(res_grad)
        resnet.fc = nn.Linear(resnet.fc.in_features, embed_size*8)
        self.resnet = resnet
        self.batch_size = batch_size
        self.embed_size = embed_size

    def forward(self, image):
        m = 8
        features = self.resnet(image)
        features = features.unsqueeze(1)
        features = features.view(-1,m,self.embed_size)
        features = features.permute(1,0,2)
        return features
"""        
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        #self.len = max_len

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
class TransformerDecoderLayer_pre(nn.TransformerDecoderLayer):
    def __init__(self, d_model, nhead):
        super().__init__(d_model=d_model, nhead=nhead)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):
        tgt_ = self.norm1(tgt)
        tgt2 = self.self_attn(tgt_, tgt_, tgt_, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt_ = self.norm2(tgt)
        tgt2 = self.multihead_attn(tgt_, memory, memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt_ = self.norm3(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt_))))
        tgt = tgt + self.dropout3(tgt2)
        #tgt = self.norm3(tgt)
        return tgt


class Decoder(nn.Module):
    def __init__(self,d_model=512,n_head=8,num_layer=6,outtoken=193,dropout=0.1,batch_size=64,length=160):
        super(Decoder, self).__init__()
        self.decoder_layer = TransformerDecoderLayer_pre(d_model=d_model, nhead=n_head)
        self.transformdecoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_layer)
        self.fc_out = nn.Linear(d_model, outtoken)
        self.decoder = nn.Embedding(outtoken, d_model)
        self.pos_decoder = PositionalEncoding(d_model, dropout,max_len=length)
        
    def forward(self,memory,tgc,tgt_mask=None):
        tgc = self.decoder(tgc)
        #print(tgc.size())
        tgc = self.pos_decoder(tgc)
        #memory = self.resn(src1,src2,src3,src4)
        #print(memory.shape)
        #print(tgc.shape)
        if tgt_mask is None:
            tgt_mask = self.generate_square_subsequent_mask(len(tgc)).to(tgc.device)
        if memory.size()[-1] == tgc.size()[-1]:
            output = self.transformdecoder(tgc,memory,tgt_mask=tgt_mask)
            output = self.fc_out(output)
        else:
            print("src and tgc dim different!")
        return output
        
    def generate_square_subsequent_mask(self, sz):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
    def predict(self,memory,tgc=None,aim_length=160):
        """ 
        the memory is 85 X 1 x d_model,output is list
        """
        epo = memory.size()[1]
        pred_num = torch.full((1,epo),word2num['<BOS>']).cuda()
        #pred = self.decoder(pred_num)
        pred_list = torch.zeros((epo,aim_length),dtype=torch.long).cuda()
        pred_list[:,0] = pred_num.reshape(-1)
        for i in range(1,aim_length):
            pred = self.decoder(pred_list[:,:i].transpose(0,1))
            #print(pred.shape)
            pred = self.pos_decoder(pred)
            #print(pred.shape)
            output = self.transformdecoder(pred,memory)
            output = self.fc_out(output)
            #print(output.size())
            #return output,pred_list
            output = output[-1].argmax(1).reshape(-1)
            #print(output.shape)
            pred_list[:,i] = output
        return pred_list
class Resnet_transdecoder(nn.Module):
    def __init__(self,d_model=512,num_layer=3,batch_size=64,length=160):
        super(Resnet_transdecoder,self).__init__()
        self.resn = EncoderCNN()
        self.decoder = Decoder(d_model=d_model,num_layer=num_layer,batch_size=batch_size,length=length)
    def forward(self,src1,tgc):
        memory = self.resn(src1)
        output = self.decoder(memory,tgc)
        return output

In [ ]:
class LabelSmoothCELoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred, label, smoothing=0.1):
        pred = F.softmax(pred, dim=1)
        one_hot_label = F.one_hot(label, pred.size(1)).float()
        smoothed_one_hot_label = (
            1.0 - smoothing) * one_hot_label + smoothing / pred.size(1)
        loss = (-torch.log(pred)) * smoothed_one_hot_label
        loss = loss.sum(axis=1, keepdim=False)
        loss = loss.mean()

        return loss

In [ ]:
from time import ctime
embedding_dim = 512
#vocab_len = 39 #39
vocab_len=193
epoches = 3
input_len = 160 # adjust small?
save_time = 1
print_time = 1000
def train(train_sample,model_path=None,model_dir='/content/',d_model=512,epoches=3,accumulation=8,batch_size=64,output_token=193,print_time=1000):
    testda = input_pic(train_sample['filepath'],train_sample['InChI_text'],maxlen=158)

    train_len = int(len(testda) * 0.9)
    val_len = len(testda) - train_len
    train_set, val_set = random_split(testda, [train_len, val_len])

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4)
    #testdata = DataLoader(testda,batch_size=batch_size,shuffle=False,num_workers=1)
    #resn = EncoderCNN(embed_size=d_model,batch_size=batch_size)
    model = Resnet_transdecoder(d_model=d_model,num_layer=1,batch_size=64,length=160)
    if model_path:
        model.load_state_dict(torch.load(model_path))
    print("train length is {}, val length is {}".format(len(train_loader),len(val_loader)))
    #resn = resn.cuda()
    model = model.cuda()
    # optimizer = optim.SGD(model.parameters(), lr=0.5)
    optimizer = optim.Adam(model.parameters(),lr=5e-3,weight_decay=1e-6)
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 4, eta_min = 1e-6, last_epoch = -1 )
    #criterion = nn.CrossEntropyLoss()
    criterion = LabelSmoothCELoss()
    best_loss = 100
    for i in range(epoches):
        model.train()
        epoch_loss = 0
        for index,(pic,lab,ind) in enumerate(train_loader):
            pic = pic.cuda()
            lab = lab.transpose(0,1).cuda()
            output = model(pic,lab[:-1,:])
            n = output.shape[-1]
            loss = criterion(output.reshape(-1, n), lab[1:, :].reshape(-1))
            loss = loss / accumulation # 累计数量梯度更新
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            if (index+1) % accumulation == 0:
                optimizer.step()
                optimizer.zero_grad()
            epoch_loss += loss.item()
            if index % print_time == 0:
                print("current error is {}".format(loss.item()))
                print("current time is {}",end=" ")
                print(ctime())
        epoch_loss_tra = epoch_loss/len(train_loader)
        print("train loss aveg batch is {}".format(epoch_loss_tra))
        model.eval()
        epoch_loss = 0
        with torch.no_grad():
            valid_dis = []
            for i, batch in enumerate(val_loader):
                pic,lab,ind = batch
                pic = pic.cuda()
                lab = lab.transpose(0,1).cuda()
                memory = model.resn(pic)
                #output = model(pic,lab[:-1,:])
                output = model.decoder(memory,lab[:-1,:])
                n = output.shape[-1]
                loss = criterion(output.reshape(-1, n), lab[1:, :].reshape(-1))
                epoch_loss += loss.item()
                textlist = model.decoder.predict(memory,aim_length=160)
                textlist = textlist.cpu().detach().numpy()
                texts = [ints_to_string_comp(text) for text in textlist]
                lab = lab.transpose(0,1).cpu().detach().numpy()
                truetext = [ints_to_string_comp(text) for text in lab]
                dis = []
                for row in range(len(texts)):
                    dis.append(Levenshtein.distance(texts[row],truetext[row]))
                valid_dis.append(np.mean(dis))
            epoch_loss_val = epoch_loss / len(val_loader)
            print('valid data loss is {}'.format(epoch_loss_val))
            print('valid data Levensh dist is {}'.format(np.mean(valid_dis)))
            if epoch_loss_val < best_loss:
                best_loss = epoch_loss_val
                model_name = model_dir+"model_{0:.5f}.pt".format(epoch_loss_val)
                print(model_name)
                torch.save(model.state_dict(), model_name)
        print('save finished')
    return model_name

In [ ]:
#import pynvml
def predict(model,idv,length = 160,epo=64,types='test'):
    n = len(idv)
    model = model.cuda()
    model.eval()
    rs_str = []
    print('test data num :{}'.format(n))
    #pynvml.nvmlInit()
    #handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    #meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
    with torch.no_grad():
        tp_list = []
        epo_num = int(n/epo)
        print('epoches num is {}'.format(epo_num))
        for i in range(epo_num+1):
            pic = torch.zeros(epo,3,224,224)
            if (i+1) * epo > n:
                for j , num_i in enumerate(range(i*epo,n)):
                    pic_id = read_pic2(idgetpath(idv[num_i],type1=types))
                    pic[j] = pic_id
                epo_act = j + 1
            else:
                for j,num_i in enumerate(range(i*epo,i*epo+epo)):
                    pic_id = read_pic2(idgetpath(idv[num_i],type1=types))
                    pic[j] = pic_id
                epo_act = epo
                print("for fin")
            if i == 0:
                print('success data')
            pic = pic[:epo_act].cuda()
            pic = pic.to(torch.float32)
            memory = model.resn(pic)
            #print(memory.size())
            #print(memory.shape)
            textlist = model.decoder.predict(memory,aim_length=length)#.transpose(0,1)
            if i == 0:
                print("success output")
            #pred_num = [word2num['<BOS>']]
            #pre = prior
            #print(textlist.shape)
            if i % 20 == 0:
                print("1 w data predict!")
            textlist = textlist.cpu().detach().numpy()
            texts = [ints_to_string_comp(text) for text in textlist]
            tp_list = tp_list + texts
    return tp_list

In [ ]:
if CFG_train:
    st = time()
    #model_path = '/kaggle/input/model-deal/model_0.38631.pt'
    mdname = train(tr_label[:1000000],model_dir='/kaggle/working/',epoches=3)
    print(time()-st)

In [ ]:
if not CFG_train:
    st = time()
    model = Resnet_transdecoder(d_model=512,batch_size=512,length=160)
    #model.load_state_dict(torch.load(mdname))
    test = pd.read_csv('sample_submission.csv')
    testeg = test[:10000]
    testdata = predict(model,testeg['image_id'],length=160,epo=512,types='test')
    x = pd.DataFrame({'image_id':testeg.image_id,'InChI':testdata})
    x.to_csv(datadir+'partdata1.csv')
    print(time()-st)